### Automação do SUAP-IFPE versão 0.1

#### Importando webdriver e abrindo SUAP

In [1]:
import pandas as pd

tombos_df = pd.read_excel('sala_aula_h_05.xlsx')
display(tombos_df.head())

tombo_impressao = []
empenho_impressao = []
uasg_impressao = []
nf_impressao = []
dtnf_impressao = []
qrcode_impressao = []

carga_impressao = []
descricao_impressao = []

,tombo
0,239277
1,241245
2,237124
3,237125
4,123531


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time



def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys("2184579")

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys("sis1986%")

    nav.find_element_by_xpath('//*[@id="ok"]').click()

    
# abrir_suap()


In [ ]:
def trocar_sala():
    
    abrir_suap()
    
    for linha in tombos_df["tombo"]:
        nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")

        tombo_impressao.append(linha)
        print(linha)

        carga = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[7]/td[2]").get_attribute('innerText')
        carga_impressao.append(carga)
        print(carga)

        descricao = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[6]/td[2]/span[1]").get_attribute('innerText')
        descricao_impressao.append(descricao)
        print(descricao)

        valor_sala_suap = nav.find_element_by_xpath('//*[@id="id_sala"]').get_attribute('value')
        valor_nova_sala = '1990'

        # time.sleep(1)

        if valor_sala_suap != valor_nova_sala:
            nova_sala = Select(nav.find_element_by_id('id_sala'))
            nova_sala.select_by_value(valor_nova_sala)
            time.sleep(1)
            nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[2]/input').click()
    
    nav.quit()

trocar_sala()

In [3]:
def impressao_etiqueta():
    
    abrir_suap()
    
    for linha in tombos_df["tombo"]:
    
        nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")
        # print(linha)
        nav.find_element_by_link_text("Detalhes da Entrada").click()
        
        tombo_impressao.append(linha)
        # print(linha)
    
        empenho = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[12]/td/a").get_attribute('innerText')
        empenho = empenho[:12]
        empenho_impressao.append(empenho)
        # print(empenho)

        uasg = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[1]/td").get_attribute('innerText')
        uasg_impressao.append(uasg)
        # print(uasg)

        nf = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td").get_attribute('innerText')
        nf_impressao.append(nf)
        # print(nf)

        data = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[5]/td").get_attribute('innerText')
        dtnf_impressao.append(data)
        # print(data)

        qrcode = str(linha)+" "+empenho+" "+" "+uasg+" "+nf+" "+data
        qrcode_impressao.append(qrcode)
        
    nav.quit()
    
impressao_etiqueta()

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=91.0.4472.124)


In [11]:
def montar_impressao_etiqueta():
    tabela_impressao = pd.DataFrame()
    tabela_impressao['patrimonio'] = tombo_impressao
    tabela_impressao['empenho'] = empenho_impressao
    tabela_impressao['uasg'] = uasg_impressao
    tabela_impressao['nota fiscal'] = nf_impressao
    tabela_impressao['data nota fiscal'] = dtnf_impressao
    tabela_impressao['qrcode'] = qrcode_impressao
    display(tabela_impressao.head())
    tabela_impressao.to_excel("sala_aula_h_05_impressao.xls", index=False, sheet_name="plan1")

montar_impressao_etiqueta()


,patrimonio,empenho,uasg,nota fiscal,data nota fiscal,qrcode
0,239277,2018NE800167,CAFI,1072,12/12/2018,239277 2018NE800167 CAFI 1072 12/12/2018
1,241245,2018NE800166,CAFI,392,08/02/2018,241245 2018NE800166 CAFI 392 08/02/2018
2,237124,2018NE800105,CAFI,3908,22/10/2018,237124 2018NE800105 CAFI 3908 22/10/2018
3,237125,2018NE800105,CAFI,3908,22/10/2018,237125 2018NE800105 CAFI 3908 22/10/2018
4,123531,2012NE800086,CAFI,2596,21/12/2012,123531 2012NE800086 CAFI 2596 21/12/2012


<ipython-input-11-5ac7cb9a1878>:10: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  tabela_impressao.to_excel("sala_aula_h_05_impressao.xls", index=False, sheet_name="plan1")


In [ ]:
def montar_tabela_carga():
    tabela_carga = pd.DataFrame()
    tabela_carga['tombo'] = tombo_impressao
    tabela_carga['descricao'] = descricao_impressao
    tabela_carga['carga'] = carga_impressao

    display(tabela_carga)

montar_tabela_carga()